In [23]:
import sys
import os
import pandas as pd
import numpy as np

project_root = r"C:\Users\esram\OneDrive\Desktop\esrabecode\immo-eliza-ml"
sys.path.append(project_root)

from scripts.clean_data import clean_immo_data

file_path = r"C:\Users\esram\OneDrive\Desktop\esrabecode\immo-eliza-ml\data\immovlan_cleaned_file_final.csv"
df = clean_immo_data(file_path)

df.head()


C:\Users\esram\OneDrive\Desktop\esrabecode\immo-eliza-ml\scripts\clean_data.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  "brussels": list(range(1000, 1300)),


Contains whitespace? False
Not matched types: ['apartment' 'land' 'undetermined-property' 'other']


,Property ID,Price,State of the property,Number of bedrooms,Livable surface,Garage,Number of garages,Type of heating,Type of glazing,Elevator,...,Total land surface,Swimming pool,type,postal_code,city,Region,province,main_type,price_per_sqm,Price_per_sqm_land
0,vbd20021,175000.0,New,0.0,51.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,studio,4000,liege,Wallonia,Liege,apartment,3431.372549,NaN
1,vbd30235,415000.0,New,1.0,70.0,1.0,NaN,NaN,NaN,1.0,...,NaN,1.0,apartment,1410,waterloo,Wallonia,Brabant Walloon,apartment,5928.571429,NaN
2,vbd46297,399000.0,NaN,2.0,129.0,1.0,NaN,Gas,Double glass,NaN,...,NaN,NaN,residence,1501,buizingen,Flanders,Brabant Flemish,house,3093.023256,NaN
3,vbd36813,229000.0,New,2.0,82.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,apartment,7000,mons,Wallonia,Hainaut,apartment,2792.682927,NaN
4,vbb60643,320000.0,New,3.0,106.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,apartment,7000,mons,Wallonia,Hainaut,apartment,3018.867925,NaN


In [24]:
import pandas as pd
import numpy as np

def clean_immo_data(file_path):

    # 1️⃣ Veri oku
    df = pd.read_csv(file_path, encoding="utf-8")
    
    # 2️⃣ Duplicate kontrolü
    df = df.drop_duplicates()
    
    # 3️⃣ Whitespace check (opsiyonel, loglama için)
    has_whitespace = df.applymap(lambda x: isinstance(x, str) and x != x.strip()).any().any()
    print("Contains whitespace?", has_whitespace)
    
    # 4️⃣ Province ve Region fonksiyonları
    provinces = {
        "brussels": list(range(1000, 1300)),
        "brabant_walloon": list(range(1300, 1500)),
        "brabant_flemish": list(range(1500, 2000)) + list(range(3000, 3500)),
        "antwerp": list(range(2000, 3000)),
        "limburg": list(range(3500, 4000)),
        "liege": list(range(4000, 4500)),
        "namur": list(range(4500, 5681)),
        "hainaut": list(range(5681, 6600)) + list(range(7000, 8000)),
        "luxembourg": list(range(6600, 7000)),
        "west_flanders": list(range(8000, 9000)),
        "east_flanders": list(range(9000, 10000))
    }
    
    def postcode_to_region(pc):
        if pd.isna(pc):
            return "Unknown"
        pc = int(pc)
        if 1000 <= pc <= 1299:
            return "Brussels"
        if 1300 <= pc <= 1499 or 4000 <= pc <= 7999:
            return "Wallonia"
        if 1500 <= pc <= 3999 or 8000 <= pc <= 9999:
            return "Flanders"
        return "Unknown"
    
    def postcode_to_province(postcode):
        for prov, codes in provinces.items():
            if postcode in codes:
                return prov.replace('_', ' ').title()
        return "Unknown"
    
    df["Region"] = df["postal_code"].apply(postcode_to_region)
    df["province"] = df["postal_code"].apply(postcode_to_province)
    
    # 5️⃣ Price per sqm hesaplamaları
    df["price_per_sqm"] = df["Price"] / df["Livable surface"]
    df['Price_per_sqm_land'] = df['Price'] / df['Total land surface']
    
    # 6️⃣ main_type normalize etme
    apartment_subtypes = ["apartment", "ground floor", "penthouse", "studio", "duplex", "loft", "triplex", "student flat"]
    house_subtypes = ["Residence", "Villa", "Mixed Building", "Master House", "Cottage", "Bungalow", "Chalet", "Mansion"]
    business_subtypes = ['commercial building', 'industrial building', 'office space', 'business surface', 'garage', 'parking']
    investment_subtypes = ['investment property']
    land_subtypes = ['land', 'development site', 'farming site', 'farmland', 'industrial ground', 'wood', 'green zone', 'recreational land', 'to parcel out site']
    
    def normalize_string(s):
        return str(s).strip().lower().replace('-', ' ')
    
    mapping = {normalize_string(x): "apartment" for x in apartment_subtypes}
    mapping.update({normalize_string(x): "house" for x in house_subtypes})
    mapping.update({normalize_string(x): "business" for x in business_subtypes})
    mapping.update({normalize_string(x): "investment property" for x in investment_subtypes})
    mapping.update({normalize_string(x): "land" for x in land_subtypes})
    
    df['main_type'] = df['type'].apply(lambda t: mapping.get(normalize_string(t), t))
    
    # Unmatched log
    unmapped = df[df['main_type'] == df['type']]['type'].unique()
    print("Not matched types:", unmapped)
    
    # 7️⃣ Dosyayı kaydet
    df.to_csv(file_path, index=False, encoding="utf-8")
    
    return df


In [25]:
file_path = r"C:\Users\esram\OneDrive\Desktop\esrabecode\immo-eliza-ml\data\immovlan_cleaned_file_final.csv"

# Temizleme fonksiyonunu çalıştır ve sonucu df olarak kaydet
df = clean_immo_data(file_path)


C:\Users\esram\AppData\Local\Temp\ipykernel_22512\880879827.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  has_whitespace = df.applymap(lambda x: isinstance(x, str) and x != x.strip()).any().any()


Contains whitespace? False
Not matched types: ['apartment' 'land' 'undetermined-property' 'other']
